In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
!killall python

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit



import tensorflow as tf

from tensorflow.contrib.rnn import LSTMCell, DropoutWrapper



import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec

"""wd_5_bigru_cnn

在论文 Recurrent Convolutional Neural Networks for Text Classification 中。

使用 BiRNN 处理，将每个时刻的隐藏状态和原输入拼起来，在进行 max_pooling 操作。

这里有些不同，首先也是使用 bigru 得到每个时刻的，将每个时刻的隐藏状态和原输入拼起来；

然后使用输入到 TextCNN 网络中。

"""





class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 15
        self.title_len = 15
        self.filter_sizes = [2, 3, 4, 5]
        self.n_filter = 200
        self.fc_hidden_size = 512
        self.n_class = 1
        self.train_batch_size=1024
        self.vaild_batch_size=5000
        self.lr=0.0025
        self.decay_step=20000
        self.decay_rate=0.8
        self.rnn_hidden_units=100
        self.rnn_hidden_size=256





class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_filter = settings.n_filter
        self.n_filter_total = self.n_filter * len(self.filter_sizes)
        self.n_class = settings.n_class
        self.fc_hidden_size = settings.fc_hidden_size
        self.rnn_hidden_units = settings.rnn_hidden_units
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        self.update_emas = list()
        # placeholders
        self.training = tf.placeholder(tf.bool)
        self._keep_prob = tf.placeholder(tf.float32, [])
        self._batch_size = tf.placeholder(tf.int32, [])
        self.embedding_size=100
        self.w=0
        self.rnn_hidden_size=settings.rnn_hidden_size

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self.query_real_length = tf.placeholder(tf.int32, [None], name='a_length')
            self.title_real_length = tf.placeholder(tf.int32, [None], name='b_length')
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')


        with tf.variable_scope('out_layer'):

            self.sim = self.esim_foward()

            self._y_pred = self.sim 
        

        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))

    @property
    def tst(self):
        return self._tst

    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step

    @property
    def X1_inputs(self):
        return self._X1_inputs

    @property
    def X2_inputs(self):
        return self._X2_inputs

    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss

    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)
    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)

    def esim_foward(self):
        a_bar, b_bar = self._inputEncodingBlock('input_encoding')
        m_a, m_b = self._localInferenceBlock(a_bar, b_bar, 'local_inference')
        logits = self._compositionBlock(m_a, m_b, self.rnn_hidden_units, 'composition')
        return logits

    # feed forward unit

    def _feedForwardBlock(self, inputs, hidden_dims, num_units, scope, isReuse = False, initializer = None):
        """
        :param inputs: tensor with shape (batch_size, 4 * 2 * hidden_size)
        :param scope: scope name
        :return: output: tensor with shape (batch_size, num_units)
        """
        with tf.variable_scope(scope, reuse = isReuse):
            if initializer is None:
                initializer = tf.random_normal_initializer(0.0, 0.1)

            with tf.variable_scope('feed_foward_layer1'):
                inputs = tf.nn.dropout(inputs, self.keep_prob)
                outputs = tf.layers.dense(inputs, hidden_dims, tf.nn.relu, kernel_initializer = initializer)
            with tf.variable_scope('feed_foward_layer2'):
                outputs = tf.nn.dropout(outputs, self.keep_prob)
                results = tf.layers.dense(outputs, num_units, tf.nn.tanh, kernel_initializer = initializer)
                return results

    # biLSTM unit
    def _biLSTMBlock(self, inputs, num_units, scope, seq_len = None, isReuse = False):
        with tf.variable_scope(scope, reuse = isReuse):
            lstmCell = LSTMCell(num_units = num_units)
            dropLSTMCell = lambda: DropoutWrapper(lstmCell, output_keep_prob = self.keep_prob)
            fwLSTMCell, bwLSTMCell = dropLSTMCell(), dropLSTMCell()
            output = tf.nn.bidirectional_dynamic_rnn(cell_fw = fwLSTMCell,
                                                     cell_bw = bwLSTMCell,
                                                     inputs = inputs,
                                                     sequence_length = seq_len,
                                                     dtype = tf.float32)
            return output
    # input encoding block ("3.1 Input Encoding" in paper)
    def _inputEncodingBlock(self, scope):
        """
        :param scope: scope name

        embedded_x_query, embedded_x_title: tensor with shape (batch_size, seq_length, embedding_size)

        :return: a_bar: tensor with shape (batch_size, seq_length, 2 * hidden_size)
                 b_bar: tensor with shape (batch_size, seq_length, 2 * hidden_size)
        """

        with tf.variable_scope(scope):
            # a_bar = BiLSTM(a, i) (1)
            # b_bar = BiLSTM(b, i) (2)
            outputsPremise, finalStatePremise = self._biLSTMBlock(self.embedded_x_query, self.rnn_hidden_units,
                                                                  'biLSTM', self.query_real_length)
            outputsHypothesis, finalStateHypothesis = self._biLSTMBlock(self.embedded_x_title, self.rnn_hidden_units,
                                                              'biLSTM', self.title_real_length,
                                                              isReuse = True)
            a_bar = tf.concat(outputsPremise, axis=2)
            b_bar = tf.concat(outputsHypothesis, axis=2)
            return a_bar, b_bar



    # local inference block ("3.2 Local Inference Modeling" in paper)
    def _localInferenceBlock(self, a_bar, b_bar, scope):
        """
        :param a_bar: tensor with shape (batch_size, seq_length, 2 * hidden_size)
        :param b_bar: tensor with shape (batch_size, seq_length, 2 * hidden_size)
        :param scope: scope name

        attentionWeights: attention matrix, tensor with shape (batch_size, seq_length, seq_length)
        attentionSoft_a, attentionSoft_b: using Softmax at two directions, tensor with shape (batch_size, seq_length, seq_length)
        a_hat, b_hat: context vectors, tensor with shape (batch_size, seq_length, 2 * hidden_size)
        a_diff, b_diff: difference of a_bar and a_hat, b_bar and b_hat, tensor with shape (batch_size, seq_length, 2 * hidden_size)
        a_mul, b_mul: hadamard product of a_bar and a_hat, b_bar and b_hat, tensor with shape (batch_size, seq_length, 2 * hidden_size)
        :return: m_a: concat of [a_bar, a_hat, a_diff, a_mul], tensor with shape (batch_size, seq_length, 4 * 2 * hidden_size)
                 m_b: concat of [b_bar, b_hat, b_diff, b_mul], tensor with shape (batch_size, seq_length, 4 * 2 * hidden_size)
        """
        with tf.variable_scope(scope):
            # e = a_bar.T * b_bar (11)
            attentionWeights = tf.matmul(a_bar, tf.transpose(b_bar, [0, 2, 1]))


            # a_hat = softmax(e) * b_bar (12)
            # b_hat = softmax(e) * a_bar (13)
            attentionSoft_a = tf.nn.softmax(attentionWeights)
            attentionSoft_b = tf.nn.softmax(tf.transpose(attentionWeights))
            attentionSoft_b = tf.transpose(attentionSoft_b)


            a_hat = tf.matmul(attentionSoft_a, b_bar)
            b_hat = tf.matmul(attentionSoft_b, a_bar)

            a_diff = tf.subtract(a_bar, a_hat)
            a_mul = tf.multiply(a_bar, a_hat)


            b_diff = tf.subtract(b_bar, b_hat)
            b_mul = tf.multiply(b_bar, b_hat)

            # m_a = [a_bar, a_hat, a_bar - a_hat, a_bar 'dot' a_hat] (14)
            # m_b = [b_bar, b_hat, b_bar - b_hat, b_bar 'dot' b_hat] (15)
            m_a = tf.concat([a_bar, a_hat, a_diff, a_mul], axis = 2)
            m_b = tf.concat([b_bar, b_hat, b_diff, b_mul], axis = 2)

            return m_a, m_b

    # composition block ("3.3 Inference Composition" in paper)
    def _compositionBlock(self, m_a, m_b, hiddenSize, scope):
        """
        :param m_a: concat of [a_bar, a_hat, a_diff, a_mul], tensor with shape (batch_size, seq_length, 4 * 2 * hidden_size)
        :param m_b: concat of [b_bar, b_hat, b_diff, b_mul], tensor with shape (batch_size, seq_length, 4 * 2 * hidden_size)
        :param hiddenSize: biLSTM cell's hidden states size
        :param scope: scope name
        outputV_a, outputV_b: hidden states of biLSTM, tuple (forward LSTM cell, backward LSTM cell)
        v_a, v_b: concate of biLSTM hidden states, tensor with shape (batch_size, seq_length, 2 * hidden_size)
        v_a_avg, v_b_avg: timestep (axis = seq_length) average of v_a, v_b, tensor with shape (batch_size, 2 * hidden_size)
        v_a_max, v_b_max: timestep (axis = seq_length) max value of v_a, v_b, tensor with shape (batch_size, 2 * hidden_size)
        v: concat of [v_a_avg, v_b_avg, v_a_max, v_b_max], tensor with shape (batch_size, 4 * 2 * hidden_size)

        :return: y_hat: output of feed forward layer, tensor with shape (batch_size, n_class)
        """
        with tf.variable_scope(scope):
            outputV_a, finalStateV_a = self._biLSTMBlock(m_a, hiddenSize, 'biLSTM')
            outputV_b, finalStateV_b = self._biLSTMBlock(m_b, hiddenSize, 'biLSTM', isReuse = True)
            v_a = tf.concat(outputV_a, axis = 2)
            v_b = tf.concat(outputV_b, axis = 2)



            # v_{a,avg} = \sum_{i=1}^l_a \frac{v_a,i}{l_a}, v_{a,max} = \max_{i=1} ^ l_a v_{a,i} (18)
            # v_{b,avg} = \sum_{j=1}^l_b \frac{v_b,j}{l_b}, v_{b,max} = \max_{j=1} ^ l_b v_{b,j} (19)
            v_a_avg = tf.reduce_mean(v_a, axis = 1)
            v_b_avg = tf.reduce_mean(v_b, axis = 1)
            v_a_max = tf.reduce_max(v_a, axis = 1)
            v_b_max = tf.reduce_max(v_b, axis = 1)

            # v = [v_{a,avg}; v_{a,max}; v_{b,avg}; v_{b_max}] (20)
            v = tf.concat([v_a_avg, v_a_max, v_b_avg, v_b_max], axis = 1)

            y_hat = self._feedForwardBlock(v, self.rnn_hidden_size, self.n_class, 'feed_forward')
            return y_hat


    def cnn_inference(self, X_inputs, n_step,name):
        """TextCNN 模型。
        Args:
            X_inputs: tensor.shape=(batch_size, n_step)
        Returns:
            title_outputs: tensor.shape=(batch_size, self.n_filter_total)
        """
        inputs = X_inputs
        inputs = tf.expand_dims(inputs, -1)
        pooled_outputs = list()
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.variable_scope(("conv-maxpool-%s" % filter_size)+name):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.n_filter]
                W_filter = self.weight_variable(shape=filter_shape, name='W_filter')
                beta = self.bias_variable(shape=[self.n_filter], name='beta_filter')
                conv = tf.nn.conv2d(inputs, W_filter, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                conv_bn=tf.layers.batch_normalization(conv,training=self.training,name=name)
                # Apply nonlinearity, batch norm scaling is not useful with relus
                # batch norm offsets are used instead of biases,使用 BN 层的 offset，不要 biases
                h = tf.nn.relu(conv_bn, name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, n_step - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1], padding='VALID', name="pool")
                pooled_outputs.append(pooled)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, self.n_filter_total])
        return h_pool_flat  # shape = [batch_size, self.n_filter_total]



def loader(data_store_path=''):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])
    print("开始加载数据")
    all_data=pd.read_pickle(data_store_path+"/home/kesci/train_data30000w-32000w.pickle")
    data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.001,random_state=1)
    train_index,vaild_index=data_split.split(all_data['label'],all_data['label']).__next__()
    train_data=all_data.iloc[train_index]
    vaild_data=all_data.iloc[vaild_index]
    print("加载数据完成")
    return train_data,vaild_data

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=True):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

from gensim import models
import pickle
fw=open('dictionary/dictionary.pickle','rb')
dictionary=pickle.load(fw)
import pickle
fw=open('tfidf_model/tfidf_model.pickle','rb')
tfidf_model=pickle.load(fw)

def get_w2v_array(word_list,max_len):
    single_sentence_tfidf=tfidf_model[dictionary.doc2bow(word_list)]
    # single_sentence_tfidf={dictionary[word_id_tfidf[0]]:word_id_tfidf[1] for word_id_tfidf in single_sentence_tfidf}
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]

                # except:
                #     pass
                
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]
                # except:
                #     pass
    return array

settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,4)

vaild_batches=np.array(vaild_data)
vaild_batches_y=vaild_batches[:,2]
vaild_batches_x=np.delete(vaild_batches, 2, axis=1)
vaild_data=batch_iter(vaild_batches_x,vaild_batches_y,settings.vaild_batch_size,1000)

print("train starting")
w2v_model = Word2Vec.load('w2v_model3/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.all_features_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)

learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                       settings.decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)

update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config) #启动创建的模型
# sess.run(tf.initialize_all_variables())

# saver = tf.train.Saver(tf.global_variables())
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
# saver.restore(sess, tf.train.latest_checkpoint('/home/kesci/work/tf_model_textcnn/'))
per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
print("start")
parts_time=[]
for batch in batches:
    from time import time
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)

    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    query_real_length = batch_x[[0]].applymap(lambda x:len(x)).values.reshape(-1)
    title_real_length = batch_x[[1]].applymap(lambda x:len(x)).values.reshape(-1)
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)

        
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.training: True, 
                 model.keep_prob: 0.8,
                 model.query_real_length: query_real_length,
                 model.title_real_length: title_real_length,
                 }
    
    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        query_real_length = vaild_x[[0]].applymap(lambda x:len(x)).values.reshape(-1)
        title_real_length = vaild_x[[1]].applymap(lambda x:len(x)).values.reshape(-1)
        vaild_x=np.array(vaild_x)
    
        #####[None,title_len,embedding_size]
        #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
        #query   title   title_count         title_ctr      label
        
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)


        # #########################这是连续型特征
        # vaild_x_ctr_fea=vaild_x[:,2:]######这里需要到时候调整，按label的位置和特征位置
        # vaild_x_ctr_fea=np.delete(vaild_x_ctr_fea, -1, axis=1)######需要把类别型特征去掉
        # ##########################
        # ###########################这是类别型特征
        # vaild_x_tfidf=vaild_x[:,-1][:,None]
        # #########################################


        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.training: False, 
                     model.keep_prob: 1,
                    model.query_real_length: query_real_length,
                    model.title_real_length: title_real_length,
                    }
        
        
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3,var_list=tf.global_variables())
        saver = saver.save(sess, "/home/kesci/work/tf_model_textcnn_esim/esim", global_step=step)

开始加载数据
加载数据完成
train starting


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


start
step 10,loss:0.478304
step 20,loss:0.482207
step 30,loss:0.48416
step 40,loss:0.48709
step 50,loss:0.489043
step 60,loss:0.457793
step 70,loss:0.480254
step 80,loss:0.48709
step 90,loss:0.483184
step 100,loss:0.486113
step 110,loss:0.477324
step 120,loss:0.470488
step 130,loss:0.483184
step 140,loss:0.486113
step 150,loss:0.463652
step 160,loss:0.470488
step 170,loss:0.495879
step 180,loss:0.469512
step 190,loss:0.508574
step 200,loss:0.469512
step 210,loss:0.491973
step 220,loss:0.488066
step 230,loss:0.483184
step 240,loss:0.49002
step 250,loss:0.478301
step 260,loss:0.485137
step 270,loss:0.463652
step 280,loss:0.475371
step 290,loss:0.48416
step 300,loss:0.468535
step 310,loss:0.480254
step 320,loss:0.48709
step 330,loss:0.477324
step 340,loss:0.479277
step 350,loss:0.461699
step 360,loss:0.48709
step 370,loss:0.479277
step 380,loss:0.500762
step 390,loss:0.48416
step 400,loss:0.48416
step 410,loss:0.472442
step 420,loss:0.48416
step 430,loss:0.488066
step 440,loss:0.475372
s

开始加载数据
加载数据完成
train starting


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 10,loss:0.482207
step 20,loss:0.482207
step 30,loss:0.509551
step 40,loss:0.51248
step 50,loss:0.49002
step 60,loss:0.500762
step 70,loss:0.516387
step 80,loss:0.508574
step 90,loss:0.496855
step 100,loss:0.509551
step 110,loss:0.488066
step 120,loss:0.497832
step 130,loss:0.511504
step 140,loss:0.48416
step 150,loss:0.482207
step 160,loss:0.469512
step 170,loss:0.501738
step 180,loss:0.482207
step 190,loss:0.485137
step 200,loss:0.507598
step 210,loss:0.471465
step 220,loss:0.514434
step 230,loss:0.497832
step 240,loss:0.501738
step 250,loss:0.500762
step 260,loss:0.501738
step 270,loss:0.498809
step 280,loss:0.483184
step 290,loss:0.506621
step 300,loss:0.506621
step 310,loss:0.482207
step 320,loss:0.489043
step 330,loss:0.501738
step 340,loss:0.477324
step 350,loss:0.496855
step 360,loss:0.492949
step 370,loss:0.506621
step 380,loss:0.503691
step 390,loss:0.507598
step 400,loss:0.478301
step 410,loss:0.48